In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:  \
           Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   
250    5.511833    0  249  114.15  595  113.63  0.50  0.03  105.53   
2750   2.237125   10  249  112.00  532  111.04  0.95  0.01  104.18   
5250   2.333756   20  249  111.47  517  110.26  1.21  0.00  103.85   
7750   2.610315   30  249  111.22  510  109.86  1.35  0.01  103.67   
10250  3.079149   40  249  111.04  505  109.57  1.46  0.01  103.53   
12750  2.725278   50  249  110.90  502  109.34  1.55  0.01  103.45   
15250  2.944422   60  249  110.78  499  109.15  1.62  0.01  103.35   
17750  2.759723   70  249  110.68  496  108.99  1.69  0.01  103.28   
20250  3.876971   80  249  110.60  494  108.84  1.75  0.00  103.19   
22750  2.851825   90  249  110.53  492  108.72  1.80  0.00  103.18   
25250  2.944527  100  249  110.46  490  108.61  1.84  0.00  103.14   
27750  2.717705  110  249  110.41  489  108.52  1.88  0.00  103.14   
30250  2.910337  120  249  110.36  487  108.44  1.91  0.00  103.11   
32750  2.718525  130  249  110.32  486  108.37  1.94  0.00  103.12   
35250  4.119220  140  249  110.28  485  108.30  1.97  0.00  103.09   
37750  3.855176  150  249  110.24  484  108.25  1.99  0.00  103.11   
40250  3.124479  160  249  110.21  484  108.19  2.01  0.00  103.08   
42750  4.064674  170  249  110.18  483  108.14  2.03  0.00  103.11   
45250  2.962970  180  249  110.15  482  108.10  2.05  0.00  103.08   
47750  3.403362  190  249  110.13  481  108.05  2.07  0.00  103.11   
50250  5.293227  200  249  110.10  481  108.01  2.09  0.00  103.07   
52750  3.036001  210  249  110.08  480  107.98  2.10  0.00  103.11   
55250  2.899807  220  249  110.06  480  107.94  2.12  0.00  103.06   
57750  2.851478  230  249  110.04  479  107.91  2.13  0.00  103.11   
60250  2.974489  240  249  110.02  479  107.88  2.14  0.00  103.06   
62750  2.780002  250  249  110.00  478  107.85  2.15  0.00  103.10   
65250  5.027911  260  249  109.99  478  107.82  2.16  0.00  103.06   
67750  2.809836  270  249  109.97  477  107.80  2.17  0.00  103.10   
70250  2.935836  280  249  109.96  477  107.77  2.18  0.00  103.05   
72750  2.775778  290  249  109.94  477  107.75  2.19  0.00  103.10   
75250  2.913146  300  249  109.93  476  107.73  2.20  0.00  103.05   
77750  2.786500  310  249  109.92  476  107.71  2.21  0.00  103.10   
80250  3.000460  320  249  109.91  476  107.69  2.22  0.00  103.04   
82750  2.750035  330  249  109.90  475  107.67  2.23  0.00  103.09   
85250  4.319331  340  249  109.88  475  107.65  2.23  0.00  103.04   
87750  2.777475  350  249  109.87  475  107.64  2.24  0.00  103.09   

                                        TEST:             SPEC:              \
              PPL     NLL    KL   REG    LOSS         PPL     1     2     3   
250    534.709167  105.17  0.36  0.00  103.65  532.194763  0.15  0.16  0.19   
2750   499.890869  102.98  1.20  0.00  102.30  496.518188  0.28  0.45  0.41   
5250   490.661041  102.42  1.43  0.00  102.09  490.134216  0.29  0.52  0.45   
7750   484.580841  102.12  1.54  0.00  101.96  485.971313  0.29  0.59  0.45   
10250  482.205780  101.84  1.69  0.00  101.86  482.659882  0.30  0.62  0.46   
12750  480.492157  101.66  1.80  0.00  101.76  480.029999  0.30  0.63  0.50   
15250  477.871582  101.43  1.91  0.00  101.62  476.786560  0.31  0.63  0.53   
17750  475.689728  101.30  1.97  0.00  101.55  474.626892  0.31  0.63  0.54   
20250  473.537048  101.15  2.04  0.00  101.47  472.912445  0.31  0.63  0.55   
22750  473.007629  101.12  2.06  0.00  101.46  472.197937  0.31  0.63  0.56   
25250  472.058350  101.03  2.11  0.00  101.41  471.239044  0.31  0.63  0.55   
27750  472.065308  101.04  2.11  0.00  101.41  471.239044  0.31  0.63  0.56   
30250  471.310944  100.96  2.15  0.00  101.38  470.236115  0.31  0.63  0.56   
32750  471.500214  100.98  2.14  0.00  101.38  470.236115  0.31  0.63  0.57   
35250  470.885468  100.92  2.18  0.00  101.36  469.624847  0.31  0.63  0.56   
37750  471.098602  100.

0 R: 1.000 P: 0.363 carry pockets strap room back nice work space lot quality
   1 R: 0.236 P: 0.106 sleeve inch protection inside zipper padding bit nice made snug
     11 R: 0.069 P: 0.069 pocket mouse power charger room netbook ipad cord small perfect
     14 R: 0.061 P: 0.061 & ; pro air protection perfectly protect chromebook inside mac
   4 R: 0.202 P: 0.096 price item amazon $ smell quality received arrived reviews time
     43 R: 0.057 P: 0.057 color love pro perfectly cover picture pink blue colors apple
     42 R: 0.049 P: 0.049 ! love perfect ... recommend bought buy absolutely awesome loves
   5 R: 0.062 P: 0.043 months zipper broke handle started strap weeks quality years back
     51 R: 0.008 P: 0.008 've year years months wear 'm bought daily hard fall
     52 R: 0.012 P: 0.012 school bought year college ... gift ! son books student
   2 R: 0.136 P: 0.070 bottom top plastic speck piece back part nice corners rubber
     21 R: 0.066 P: 0.066 cover keyboard mac hard easy s